<a href="https://colab.research.google.com/github/mr-cri-spy/LLM-Playground/blob/main/8_bit_Training_for_Massive_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch==2.4.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [4]:
import torch
# Check so there is a gpu available, a T4(free tier) will NOT be enough for an 14B parameter model, but we can use a slightly smaller one and the lesson will remain the same
assert (torch.cuda.is_available()==True)

AssertionError: 

In [5]:
!pip install -e 'git+https://github.com/axolotl-ai-cloud/axolotl.git@78b42a3fe13c49e317bc116b9999c30e070322cc#egg=axolotl' # ensures the same version we used in the course
#!pip install flash-attn==2.6.0 # needed if using Phi-3

Obtaining axolotl from git+https://github.com/axolotl-ai-cloud/axolotl.git@78b42a3fe13c49e317bc116b9999c30e070322cc#egg=axolotl
  Cloning https://github.com/axolotl-ai-cloud/axolotl.git (to revision 78b42a3fe13c49e317bc116b9999c30e070322cc) to ./src/axolotl
  Running command git clone --filter=blob:none --quiet https://github.com/axolotl-ai-cloud/axolotl.git /content/src/axolotl
  Running command git rev-parse -q --verify 'sha^78b42a3fe13c49e317bc116b9999c30e070322cc'
  Running command git fetch -q https://github.com/axolotl-ai-cloud/axolotl.git 78b42a3fe13c49e317bc116b9999c30e070322cc
  Running command git checkout -q 78b42a3fe13c49e317bc116b9999c30e070322cc
  Resolved https://github.com/axolotl-ai-cloud/axolotl.git to commit 78b42a3fe13c49e317bc116b9999c30e070322cc
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/transformers.git (to revision 026a173a64372e9602a16523b8fae9de4b0ff428) to /tmp/pip-install-ybtvzfs5/transformers_0bef8cd7c16e4d0b97bd6c4de3

In [6]:
import yaml

train_config = {
    # "base_model": "microsoft/Phi-3-meidum-128k-instruct" # this requires a 24GB video card
    "base_model": "unsloth/llama-3-8b-Instruct", # this can be run on the free tier of Colab


    # dataset params
    "datasets": [
        {
            "path": "TheFuzzyScientist/squad-for-llms",
            "type": {
                "system_prompt": "Read the following context and concisely answer my question.",
                "field_system": "system",
                "field_instruction": "question",
                "field_input": "context",
                "field_output": "output",
                "format": "<|user|> {input} {instruction} </s> <|assistant|>",
                "no_input_format": "<|user|> {instruction} </s> <|assistant|>",
            },
        }
    ],
    "output_dir": "./models",

    # model params
    "sequence_length": 8172,
    "bf16": "auto",
    "tf32": False,

    # training params
    "micro_batch_size": 4,
    "num_epochs": 1,
    "optimizer": "adamw_bnb_8bit",
    "learning_rate": 0.0002,
    "logging_steps": 1,

    # LoRA
    "adapter": "lora",
    "lora_r": 32,
    "lora_alpha": 16,
    "lora_dropout": 0.05,
    "lora_target_linear": True,

    # Gradient Accumulation
    "gradient_accumulation_steps": 1,

    # Gradient Checkpointing
    "gradient_checkpointing": True,

    # Low Precision
    "load_in_8bit": True,
}

# Write the YAML file
with open("specialised_train.yml", 'w') as file:
    yaml.dump(train_config, file)


In [7]:
!accelerate launch -m axolotl.cli.train specialised_train.yml

Traceback (most recent call last):
  File "/usr/local/bin/accelerate", line 4, in <module>
    from accelerate.commands.accelerate_cli import main
  File "/usr/local/lib/python3.12/dist-packages/accelerate/commands/accelerate_cli.py", line 19, in <module>
    from accelerate.commands.estimate import estimate_command_parser
  File "/usr/local/lib/python3.12/dist-packages/accelerate/commands/estimate.py", line 35, in <module>
    import timm
  File "/usr/local/lib/python3.12/dist-packages/timm/__init__.py", line 2, in <module>
    from .layers import (
  File "/usr/local/lib/python3.12/dist-packages/timm/layers/__init__.py", line 1, in <module>
    from ._fx import (
  File "/usr/local/lib/python3.12/dist-packages/timm/layers/_fx.py", line 8, in <module>
    from torchvision.models.feature_extraction import create_feature_extractor as _create_feature_extractor
  File "/usr/local/lib/python3.12/dist-packages/torchvision/__init__.py", line 10, in <module>
    from torchvision import _meta_

In [8]:
# Optional: Merge the trained adapter
!accelerate launch -m axolotl.cli.merge_lora specialised_train.yml

Traceback (most recent call last):
  File "/usr/local/bin/accelerate", line 4, in <module>
    from accelerate.commands.accelerate_cli import main
  File "/usr/local/lib/python3.12/dist-packages/accelerate/commands/accelerate_cli.py", line 19, in <module>
    from accelerate.commands.estimate import estimate_command_parser
  File "/usr/local/lib/python3.12/dist-packages/accelerate/commands/estimate.py", line 35, in <module>
    import timm
  File "/usr/local/lib/python3.12/dist-packages/timm/__init__.py", line 2, in <module>
    from .layers import (
  File "/usr/local/lib/python3.12/dist-packages/timm/layers/__init__.py", line 1, in <module>
    from ._fx import (
  File "/usr/local/lib/python3.12/dist-packages/timm/layers/_fx.py", line 8, in <module>
    from torchvision.models.feature_extraction import create_feature_extractor as _create_feature_extractor
  File "/usr/local/lib/python3.12/dist-packages/torchvision/__init__.py", line 10, in <module>
    from torchvision import _meta_